# Installations

In [1]:
!pip install gtts
!pip install streamlit pyngrok --quiet
!pip install reportlab
!pip install gTTS
!apt-get install -y ffmpeg
!pip install edge-tts pillow
!pip install fpdf 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:0000:0100:01
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 134 not upgraded.
  Preparing metadata (setu

In [7]:
# Sari purani memory clear karne ke liye
# !fuser -k 8501/tcp
# !pkill -f streamlit
!rm -rf /kaggle/working/*

In [ ]:

import time
import threading
import torch
from pyngrok import ngrok 
from diffusers import StableDiffusionXLPipeline
from google import genai
import re
import os
from PIL import Image, ImageDraw, ImageFont
import textwrap
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from gtts import gTTS
import subprocess
from gradio import themes
import gradio as gr

# ------------------ 1. NGROK SETUP ------------------
NGROK_TOKEN = "token"  # Reset if needed
ngrok.set_auth_token(NGROK_TOKEN)
ngrok.kill()
try:
    os.system("pkill -f gradio")
except:
    pass

API_KEY = "token"
client = genai.Client(api_key=API_KEY)

# ------------------ 2. IMPORT YOUR EXISTING LOGIC ------------------

def load_pipe():
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to("cuda")

    pipe.enable_model_cpu_offload()
    return pipe   # ✅ MOST IMPORTANT

# Load once globally
pipe = load_pipe()
# ------------------ 2. FORMAL LETTERING ------------------
def add_formal_elements(image, caption_text, dialogue_text):
    img = image.convert("RGBA")
    w, h = img.size
    draw = ImageDraw.Draw(img)
    
    caption_text = caption_text.replace("*", "").strip()
    dialogue_text = dialogue_text.replace("*", "").strip()

    try:
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
        f_cap = ImageFont.truetype(font_path, 22)
        f_dia = ImageFont.truetype(font_path, 20)
    except:
        f_cap = f_dia = ImageFont.load_default()

    if caption_text and caption_text.lower() != "none":
        wrapper = textwrap.TextWrapper(width=60)
        lines = wrapper.wrap(text=caption_text)
        h_box = (len(lines) * 30) + 20
        draw.rectangle([10, 10, w-10, h_box+10], fill=(255, 245, 150, 255), outline=(0,0,0), width=2)
        for i, line in enumerate(lines):
            draw.text((25, 15 + i*30), line, font=f_cap, fill=(0,0,0))

    if dialogue_text and dialogue_text.lower() != "none":
        formatted_dia = dialogue_text.replace(":", ":\n")
        wrapper = textwrap.TextWrapper(width=50, replace_whitespace=False)
        lines = []
        for segment in formatted_dia.split('\n'):
            lines.extend(wrapper.wrap(segment))
            
        h_box = (len(lines) * 28) + 30
        draw.rectangle([30, h-h_box-30, w-30, h-30], fill=(255, 255, 255, 250), outline=(0,0,0), width=2)
        for i, line in enumerate(lines):
            draw.text((45, h-h_box-20 + i*28), line, font=f_dia, fill=(0,0,0))
            
    return img.convert("RGB")
# ------------------ 3. VIDEO ENGINE ------------------
def create_video_with_audio(title_img, title_text, final_images, script, total_panels, video_filename):
    video_clips = []
    pause_duration = 0.8 

    def get_audio_duration(file_path):
        cmd = f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 {file_path}"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        return float(result.stdout.strip())

    title_audio = "title_audio.mp3"
    gTTS(text=f"Welcome to {title_text}", lang='en').save(title_audio)
    title_img.save("title_frame.jpg")
    
    a_dur = get_audio_duration(title_audio)
    total_dur = a_dur + pause_duration
    
    cmd_title = (
        f"ffmpeg -y -loop 1 -i title_frame.jpg -i {title_audio} "
        f"-filter_complex '[1:a]apad=pad_dur={pause_duration}[a]' "
        f"-map 0:v -map '[a]' -c:v libx264 -t {total_dur} "
        f"-pix_fmt yuv420p title_clip.mp4"
    )
    subprocess.run(cmd_title, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    video_clips.append("title_clip.mp4")

    for i in range(1, total_panels + 1):
        pattern = f"PANEL {i}:?.*?DIALOGUE:(.*?)(?=PANEL {i+1}:?|$)"
        match = re.search(pattern, script, re.DOTALL | re.IGNORECASE)
        dialogue = match.group(1).strip() if (match and match.group(1).strip()) else "Next slide."

        temp_audio = f"temp_v_{i}.mp3"
        gTTS(text=dialogue, lang='en').save(temp_audio)
        
        img_file = f"t_{i}.jpg"
        final_images[i-1].save(img_file)
        
        a_dur = get_audio_duration(temp_audio)
        total_dur = a_dur + pause_duration
        
        clip_file = f"c_{i}.mp4"
        cmd = (
            f"ffmpeg -y -loop 1 -i {img_file} -i {temp_audio} "
            f"-filter_complex '[1:a]apad=pad_dur={pause_duration}[a]' "
            f"-map 0:v -map '[a]' -c:v libx264 -t {total_dur} "
            f"-r 24 -pix_fmt yuv420p {clip_file}"
        )
        subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        video_clips.append(clip_file)

    with open("list.txt", "w") as f:
        for c in video_clips: f.write(f"file '{c}'\n")
            
    subprocess.run(f"ffmpeg -y -f concat -safe 0 -i list.txt -c:v libx264 -c:a aac -pix_fmt yuv420p {video_filename}", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return video_filename



def start_production(topic, chars, total_panels):
    script_prompt = f"""
    Create a {total_panels}-panel comic script about '{topic}'.
    IMPORTANT: The characters MUST look consistent in every panel based on this description: {chars}.
    STORY_TITLE: [generate title{topic}], TITLE_IMAGE_PROMPT: [Detailed prompt for Stable Diffusion XL ]
    
    For each panel, follow this format EXACTLY:
    PANEL [number]:
    CAPTION: [Brief narrative text for the panel]
    IMAGE_PROMPT: [Detailed prompt for Stable Diffusion XL including {chars},must in 2D, environment, and style ]
    DIALOGUE: [Character names and their speech]


    """
    # pipe = load_pipe()  # from your existing code
    script = client.models.generate_content(model="gemini-3-flash-preview", contents=script_prompt).text
 

    title_text = re.search(r"STORY_TITLE:(.*)", script, re.I).group(1).strip()
    title_prompt = re.search(r"TITLE_IMAGE_PROMPT:(.*?)PANEL 1", script, re.S | re.I).group(1).strip()
    safe_name = title_text 
    print(title_text,"\n")
    title_img = pipe(
                prompt=title_prompt, 
                negative_prompt="3d,relaistic watermark, blurry, low quality, distorted, words, letters",
                num_inference_steps=40,
                guidance_scale=8.0
            ).images[0]
            
    # ------------------ Overlay Title Text ------------------
    draw = ImageDraw.Draw(title_img)
    
    # Try using a bold system font, fallback to default if not found
    try:
        # Arial Bold font with bigger size (e.g., 120)
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
        font = ImageFont.truetype(font_path, 32)
    except:
        font = ImageFont.load_default()
    w, h = title_img.size
    
    # Create drawing context
    draw = ImageDraw.Draw(title_img)
    
    # Load font
    try:
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
        font = ImageFont.truetype(font_path, 32)
    except:
        font = ImageFont.load_default()
    
    # Convert title to uppercase
    text = title_text.upper()
    
    # Wrap text
    wrapper = textwrap.TextWrapper(width=20)
    lines = wrapper.wrap(text=text)
    
    # Calculate total height using textbbox
    bbox_single = draw.textbbox((0, 0), "A", font=font)
    line_height = bbox_single[3] - bbox_single[1] + 10  # padding
    total_height = len(lines) * line_height + 40
    
    # Draw semi-transparent grey rectangle centered
    rect_x0, rect_y0 = 50, (h - total_height) // 2
    rect_x1, rect_y1 = w - 50, rect_y0 + total_height
    draw.rectangle([rect_x0, rect_y0, rect_x1, rect_y1], fill=(200, 200, 200, 200))
    
    # Draw each line centered horizontally
    for i, line in enumerate(lines):
        bbox_line = draw.textbbox((0, 0), line, font=font)
        text_w = bbox_line[2] - bbox_line[0]
        text_x = (w - text_w) // 2
        text_y = rect_y0 + 20 + i * line_height
        draw.text((text_x, text_y), line, font=font, fill=(0, 0, 0, 255))
    # Save to check
    title_img.save("t_img.jpg")
    print("images in progress")
    final_images = []
    for i in range(1, total_panels + 1):
        pattern = f"PANEL {i}:?.*?CAPTION:(.*?)IMAGE_PROMPT:(.*?)DIALOGUE:(.*?)(?=PANEL {i+1}:?|$)"
        match = re.search(pattern, script, re.S | re.I)
        print(f"🎬 Rendering Panel {i}/{total_panels}...")
        if match:
            cap, p_prompt, dia = match.group(1).strip(), match.group(2).strip(), match.group(3).strip()
           
            img = pipe(
                prompt=f"{p_prompt}, 2D cartoon, clean lines, flat colors",
                negative_prompt="text, watermark, blurry, low quality, distorted, words, letters",
                num_inference_steps=40,
                guidance_scale=8.0
            ).images[0]
            
            final_images.append(add_formal_elements(img, cap, dia))

    # Save PDF
    print("pdf")
    pdf_filename = f"{safe_name}.pdf"
    c = canvas.Canvas(pdf_filename, pagesize=A4)
    tw, th = A4
    # title_img.save("t_img.jpg")
    c.drawImage("t_img.jpg", 0, 0, width=tw, height=th)
    c.showPage()
    for i, img in enumerate(final_images):
        img.save(f"p_{i}.jpg")
        c.drawImage(f"p_{i}.jpg", 0, 0, width=tw, height=th)
        c.showPage()
    c.save()
    print("video")
    # # Create Video
    # video_filename = create_video_with_audio(title_img, title_text, final_images, script, total_panels, f"{title_img}.mp4")
    safe_video_name = re.sub(r'[^a-zA-Z0-9_-]', '_', title_text) + ".mp4"
    video_filename = create_video_with_audio(title_img, title_text, final_images, script, total_panels, safe_video_name)
    return pdf_filename, video_filename

# For example, pipe initialization:

# ------------------ 3. GRADIO INTERFACE ------------------
def generate_comic(topic, char_desc, num_panels):
    pdf_path, vid_path = start_production(topic, char_desc, int(num_panels))

    
    return vid_path,pdf_path
    
inputs = [
    gr.Textbox(label="Prompt", placeholder="e.g., A brave knight"),
    gr.Textbox(label="Character Description", placeholder="e.g., Tall knight in gold armor"),
    gr.Slider(1, 10, value=3, step=1, label="Number of Panels")
]

outputs = [
    gr.Video(label="Watch Video"),
    gr.File(label="Download PDF", file_types=[".pdf"])
]

with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Interface(
        fn=generate_comic,
        inputs=inputs,
        outputs=outputs,
        title="TaleWeaver",
        description="Generate a comic book with AI (PDF + Video)."
    )

# ------------------ 4. LAUN CH GRADIO + NGROK ------------------
def run_gradio():
    # demo.launch(server_port=8501, share=False)  # local port, ngrok will tunnel
    
    demo.launch(server_port=8501, share=False)

thread = threading.Thread(target=run_gradio)
thread.daemon = True
thread.start()

print("⏳ Waiting  to start...")
time.sleep(8)

public_url = ngrok.connect(8501, "http")
print("\n" + "="*30)
print(f"✅ SUCCESS! Access your app here:\n{public_url}")
print("="*30)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/517 [00:00<?, ?it/s]

/tmp/ipykernel_55/3467119456.py:292: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
Exception in thread Thread-30 (run_gradio):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_55/3467119456.py", line 305, in run_gradio
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2720, in launch
    ) = http_server.start_server(
        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/http_server.py", line 157, in start_server
    raise OSError(
OSError: Cannot find empty port in range: 8501-8501. You can specify a different port by setting the GRADIO_SERVER_PORT environment vari

⏳ Waiting  to start...

✅ SUCCESS! Access your app here:
NgrokTunnel: "https://unroaming-aquatically-leonia.ngrok-free.dev" -> "http://localhost:8501"
The Secret of the Glowing Tome 



...

  0%|          | 0/40 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['colors']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['colors']


🎬 Rendering Panel 4/4...


  0%|          | 0/40 [00:00<?, ?it/s]

pdf
video
